In [1]:
# import the necessary packages
import face_recognition 
import argparse
import imutils
import pickle
import time  
import cv2
import os
import pytube

In [2]:
print("Enter Youtube Link: ")
link = str(input())
#link = https://www.youtube.com/watch?v=6lOxW0AB958

Enter Youtube Link: 
https://www.youtube.com/watch?v=6lOxW0AB958


In [3]:
#downloading the video 
print("[INFO] : This may take a couple of minutes depending upon Internet Connection...")
yt = pytube.YouTube(link)
video_title =  yt.title
print("[PROCESS] : FETCHING... ", video_title)

video = yt.streams.filter(file_extension='mp4', res="720p").first() #download('output/')
video_path = video.download('temp/')
print("[PROCESS] : FETCHED... ", video_title)

[INFO] : This may take a couple of minutes depending upon Internet Connection...


RegexMatchError: regex pattern (\W[\'"]?t[\'"]?: ?[\'"](.+?)[\'"]) had zero matches

In [ ]:
args = { 'detection_method': 'hog',
         'display': 1,
         'encodings': '/home/jugal/Desktop/Face Recognition/encodings_international_celebrity.pickle',
         'input': video_path,
         #'output':video_path[0:-4]+"-Processed.mp4"
         'output': None
       }

In [ ]:
# load the known faces and embeddings
print("[INFO] : loading encodings...")
data = pickle.loads(open(args["encodings"], "rb").read())

# initialize the pointer to the video file and the video writer
print("[PROCESS] : processing fetched contents ...")
stream = cv2.VideoCapture(args["input"])
writer = None
print("[INFO] : Starting Live Stream Face Recognition on Video")
# loop over frames from the video file stream
frame_count = 0
while True:
    
    # grab the next frame
    (grabbed, frame) = stream.read()
    # if the frame was not grabbed, then we have reached the
    # end of the stream
    
    if not grabbed:
        break

    # convert the input frame from BGR to RGB then resize it to have
    # a width of 750px (to speedup processing)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    #rgb = cv2.resize(rgb,None,fx=0.4,fy=0.4)
    #rgb = imutils.resize(rgb, width=450)
    r = frame.shape[1] / float(rgb.shape[1])

    # detect the (x, y)-coordinates of the bounding boxes
    # corresponding to each face in the input frame, then compute
    # the facial embeddings for each face
    boxes = face_recognition.face_locations(rgb,number_of_times_to_upsample=1, model=args["detection_method"])
    encodings = face_recognition.face_encodings(rgb, boxes)
    names = []

    # loop over the facial embeddings
    for encoding in encodings:
        # attempt to match each face in the input image to our known
        # encodings
        matches = face_recognition.compare_faces(data["encodings"], encoding, tolerance=0.5)
        name = "Unknown"

        # check to see if we have found a match
        if True in matches:
            # find the indexes of all matched faces then initialize a
            # dictionary to count the total number of times each face
            # was matched
            matchedIdxs = [i for (i, b) in enumerate(matches) if b]
            counts = {}

            # loop over the matched indexes and maintain a count for
            # each recognized face face
            for i in matchedIdxs:
                name = data["names"][i]
                counts[name] = counts.get(name, 0) + 1

            # determine the recognized face with the largest number
            # of votes (note: in the event of an unlikely tie Python
            # will select first entry in the dictionary)
            name = max(counts, key=counts.get)
        
        # update the list of names
        names.append(name)

    # loop over the recognized faces
    for ((top, right, bottom, left), name) in zip(boxes, names):
        # rescale the face coordinates
        top = int(top * r)
        right = int(right * r)
        bottom = int(bottom * r)
        left = int(left * r)

        # draw the predicted face name on the image
        cv2.rectangle(frame, (left, top), (right, bottom), (255, 255, 255), 2)
        y = top - 15 if top - 15 > 15 else top + 15
        cv2.putText(frame, name, (left, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    # if the video writer is None *AND* we are supposed to write
    # the output video to disk initialize the writer
    if writer is None and args["output"] is not None:
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter(args["output"], fourcc, 24, (frame.shape[1], frame.shape[0]), True)

    # if the writer is not None, write the frame with recognized
    # faces t odisk
    if writer is not None:
        writer.write(frame)

    # check to see if we are supposed to display the output frame to
    # the screen
    if args["display"] > 0:
        cv2.imshow(video_title, frame)
        key = cv2.waitKey(1) & 0xFF

        # if the `q` key was pressed, break from the loop
        if key == ord("q"):
            break
            
# close the video file pointers
stream.release()

# check to see if the video writer point needs to be released
if writer is not None:
    writer.release()
    
print("[PROCESS] : Cleaning up disk")
#os.remove(args["output"])
os.remove(args["input"])
print("[DONE]")

In [ ]:
import os
os._exit(0)